In [16]:
import plotly.express as px
import matplotlib as plt
import seaborn as sns
import pandas as pd
import numpy as np
import json
import ast


def read(path):
    with open(path, 'r') as f:
        return f.read()

    
def parse_purchase(purchases_str):
    values = []
    for purchase in purchases_str:
        value_str = purchase.split("€")[1]
        values.append(int(value_str.replace(",", "")))
    return values

In [17]:
log = read("../data/ranking.log")
users_info = pd.DataFrame([ast.literal_eval(row) 
                           for row in log.split("\n") 
                           if row and row[0] == "{"])

users_info["value"] = users_info.value.apply(lambda x: int(x[1:].replace(",", "")))
users_info["purchase_info"] = users_info.purchase_info.apply(parse_purchase)

In [22]:
users_info

,name,players,points,purchase_info,value
0,FC Datalona,"[Ter Stegen, Aitor Fernández, Sergi Roberto, H...",0,"[6630658, 1800543, 5200528, 5655110, 5058060, ...",39810000
1,U. B. Conquense,"[Rubén Blanco, Zaldua, Odriozola, Mathías Oliv...",0,"[1250300, 2910000, 480000, 380000, 1280000, 50...",39080000
2,Thomas Party,"[Vaclík, Oier Olazábal, Felipe Monteiro, Pedra...",0,"[6200000, 1300000, 4510000, 3400000, 3200000, ...",25580000
3,Camping Ballena Alegre,"[Damián Suárez, Mario Gaspar, Orellana, Saúl, ...",0,"[5560000, 660100, 6500005, 7520000, 4500000, 5...",25410000
4,Pastafarianism United,"[Cillessen, Germán Sánchez, Arbilla, Rodrigo E...",0,"[6820023, 1310003, 610022, 260001, 150051, 153...",21530000
5,Vetusta FC,"[Herrerín, Garay, Yeray, Capa, Cabaco, Aleix F...",0,"[2099000, 5430000, 3590000, 930100, 210000, 27...",14030000
6,Santa Bàrbara,"[Manolo Reina, Ximo Navarro, David Costas, Jos...",0,"[1180000, 2614990, 709500, 150000, 5089090, 67...",12740000
7,ValeronBalonDeOro,"[Rubén, Wass, Diakhaby, Salva Sevilla, Vesga, ...",0,"[1501000, 4100000, 276000, 2000000, 230000, 22...",6400000


In [25]:
players = []
for _, user in users_info.iterrows():
    players.extend(user.players)
str(players)

"['Ter Stegen', 'Aitor Fernández', 'Sergi Roberto', 'Hugo Mallo', 'Ferland Mendy', 'Lucas Vázquez', 'Roque Mesa', 'Barrenetxea', 'Fede Valverde', 'Borja Iglesias', 'Vinícius', 'Dembélé', 'Rubén Blanco', 'Zaldua', 'Odriozola', 'Mathías Olivera', 'Portillo', 'Fran Beltrán', 'Riqui Puig', 'Messi', 'Rodrigo', 'Gameiro', 'Sobrino', 'Vaclík', 'Oier Olazábal', 'Felipe Monteiro', 'Pedraza', 'Renan Lodi', 'Aridane Hernández', 'James Rodríguez', 'Óliver Torres', 'Iturraspe', 'Portu', 'Luuk de Jong', 'Damián Suárez', 'Mario Gaspar', 'Orellana', 'Saúl', 'Banega', 'William Carvalho', 'Granero', 'Kang-in Lee', 'Recio', 'Ekambi', 'Wu Lei', 'Cillessen', 'Germán Sánchez', 'Arbilla', 'Rodrigo Ely', 'Francis', 'Yokuslu', 'Szymanowski', 'Pione Sisto', 'Iago Aspas', 'Herrerín', 'Garay', 'Yeray', 'Capa', 'Cabaco', 'Aleix Febas', 'Cáseres', 'Lucas Ocampos', 'Lago Junior', 'Manolo Reina', 'Ximo Navarro', 'David Costas', 'José Antonio Martínez', 'Marc Roca', 'Arthur', 'Aleña', 'Ferrán Torres', 'Sandro', 'Beauv